In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
from datetime import datetime

In [ ]:
def time_ini():
    return datetime.now()
def time_fin(i):
    tim = datetime.now() - i
    return 'Time: {}'.format(tim).split('.')[0]

In [ ]:
import subprocess

# descargar el módulo Pandas
import pandas as pd

### Descargar la base (rdp_species_assignment_18.fa.gz) de datos 16S a partir de RDP taxonomic:

https://zenodo.org/record/4310151

### Descomprimir el archivo .gz

### Construir una base de datos

In [ ]:
subprocess.call('makeblastdb -in DB/rdp_species_assignment_18.fa -dbtype nucl -parse_seqids -out DB/species_16S', shell = True)

### Correr Blastn localmente

In [ ]:
init = time_ini()
#---------------
subprocess.call('blastn -db DB/species_16S -query bacterias.fasta -evalue 1E-6 -outfmt "6 qacc sacc qlen slen length qstart qend sstart send score bitscore evalue pident nident mismatch positive gaps gapopen" -max_target_seqs 1000 -max_hsps 1000 -out resultado.txt', shell = True)   
#----------------
time_fin(i = init)

### Abrimos los resultados del blastn

In [ ]:
columna=["qacc", "sacc", "qlen", "slen", "length", "qstart", "qend", "sstart", "send", "score", "bitscore", "evalue", "pident", "nident", "mismatch", "positive", "gaps", "gapopen"]

In [ ]:
df = pd.read_csv('resultado.txt',sep='\t',names=columna)
df

In [ ]:
"""
Función para abrir archivos fasta
"""
def open_file(file = ''):
    fas = {}
    with open(file) as fq:
        for line in fq:
            line = line.rstrip()
            if '>' in line:
                header = line.replace('>','')
                s = ''
            else:
                s += line
            fas[header] =  s
    return fas

In [ ]:
fasta = open_file(file = 'DB/rdp_species_assignment_18.fa')

In [ ]:
nombres = {}
for i in fasta.keys():
    i = i.split(' ')
    nombres[i[0]] = i[1]

In [ ]:
nombres

In [ ]:
df['Species'] = [nombres[i] for i in df.sacc]

In [ ]:
df

In [ ]:
from collections import Counter, OrderedDict

In [ ]:
df.qacc.unique()

In [ ]:
df1 = df[df.qacc == 'ASV1'].reset_index(drop = True)

In [ ]:
orden = dict(Counter(df1.Species.tolist()))
orden

In [ ]:
for i in list(reversed(sorted(set(orden.values())))):
    for x in orden:
        if orden[x] == i:
            print(i, x)

In [ ]:
for i in df.qacc.unique():
    df1 = df[df.qacc == i].reset_index(drop = True)
    orden = dict(Counter(df1.Species.tolist()))
    
    k = []
    for v in list(reversed(sorted(set(orden.values())))):
        for x in orden:
            if orden[x] == v:
                k.append([v, x])
                
    print(i, k[0])

### Contra la base de datos 16S_ribosomal_RNA del NCBI

In [ ]:
init = time_ini()
#---------------
subprocess.call('blastn -db DB/16S_ribosomal_RNA -query bacterias.fasta -evalue 1E-6 -outfmt "6 qacc sacc qlen slen length qstart qend sstart send score bitscore evalue pident nident mismatch positive gaps gapopen" -max_target_seqs 1000 -max_hsps 1000 -out resultado2.txt', shell = True)   
#----------------
time_fin(i = init)

In [ ]:
df2 = pd.read_csv('resultado2.txt',sep='\t',names=columna)
df2

In [ ]:
len(df2.sacc), len(df2.sacc.unique())

In [ ]:
subprocess.call('blastdbcmd -db DB/16S_ribosomal_RNA -entry all', shell = True)

In [ ]:
rec = []
for i in df2.sacc.unique()[:5]:
    info10 = subprocess.Popen('blastdbcmd -db DB/16S_ribosomal_RNA -entry ' + i,
                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    info100 = info10.communicate()[0].decode()
    rec.append(info100)

In [ ]:
rec